In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.dataset_download_files('asdasdasasdas/garbage-classification', path="./garbage1", quiet=False, unzip=True)
api.dataset_download_files('mostafaabla/garbage-classification', path="./garbage2", quiet=False, unzip=True)

ModuleNotFoundError: No module named 'kaggle'

In [4]:
dataset1_directory='garbage1/Garbage classification/Garbage classification/'
dataset2_directory='garbage2/garbage_classification/'

In [5]:
import shutil
import os
removable_classes=['battery', 'clothes', 'biological', 'shoes']
for label in removable_classes:
    shutil.rmtree(os.path.join(dataset2_directory,label))
os.rename('garbage2/garbage_classification/brown-glass', 'garbage2/garbage_classification/glass')
glasses = [os.path.join(dataset2_directory,'white-glass'), os.path.join(dataset2_directory,'green-glass')]
for glass_directory_name in glasses:
    for filename in os.listdir(glass_directory_name):
        shutil.move(os.path.join(glass_directory_name, filename), os.path.join('garbage2/garbage_classification/glass', filename))
    os.rmdir(glass_directory_name)

FileNotFoundError: [WinError 3] A rendszer nem találja a megadott elérési utat: 'garbage2/garbage_classification/battery'

In [6]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image as image_utils
import tensorflow as tf
import numpy as np

class_names=['glass', 'paper', 'cardboard', 'trash', 'metal', 'plastic']
image_size=(256, 256)
validation_split=0.2
seed=111
batch_size=32

In [7]:
train_1 = keras.utils.image_dataset_from_directory(
    dataset1_directory,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    seed=seed,
    subset='training'
)
val_1 = keras.utils.image_dataset_from_directory(
    dataset1_directory,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    seed=seed,
    subset='validation'
)
    
train_2 = keras.utils.image_dataset_from_directory(
    dataset2_directory,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    seed=seed,
    subset='training'
)
val_2 = keras.utils.image_dataset_from_directory(
    dataset2_directory,
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    seed=seed,
    subset='validation'
)

Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.
Found 6283 files belonging to 6 classes.
Using 5027 files for training.
Found 6283 files belonging to 6 classes.
Using 1256 files for validation.


In [8]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_1 = train_1.map(lambda x, y: (normalization_layer(x), y))
val_1 = val_1.map(lambda x, y: (normalization_layer(x), y))
train_2 = train_2.map(lambda x, y: (normalization_layer(x), y))
val_2 = val_2.map(lambda x, y: (normalization_layer(x), y))

In [9]:
train = train_1.concatenate(train_2)
val = val_1.concatenate(val_2)

In [10]:
train

<ConcatenateDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>

In [11]:
train_1

<MapDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 6), dtype=tf.float32, name=None))>

In [12]:
image = train_1.take(1)

In [ ]:
#y_train = np.concatenate([y for x, y in train], axis=0)
x_train = np.empty(shape=(0,256,256,3))
for image, label in train:
    x_train = np.concatenate((x_train, image.numpy()), axis=0)

In [113]:
print(y_train)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [130]:
for element in train.as_numpy_iterator():
    np.concatenate((x_train, element), axis=0)

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


ValueError: could not broadcast input array from shape (32,256,256,3) into shape (32,)

<MapDataset element_spec=TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)>
